- [ ] Create product subdir tree 
- [ ] Download all files to subdirs
    - [x] List all files
    - [x] Parse location from safe dir 
    - [ ] move all files to their local dir
- [ ] ID burst overlapping with AOI
    - [ ] Use burst API? Get IDs and products
    - [ ] Or use burst info in product
- [ ] Compute burst offsets
- [ ] Open datasets and add a burst_start and burst_end attribute
- [ ] Download the burst range in each IW to zarr

In [ ]:
from pystac_client.client import Client
import geopandas as gpd
import rioxarray as riox
import xarray as xr
import json
from dask.diagnostics import ProgressBar
from pathlib import Path
from urllib.parse import urlparse
import os

from rasterio.session import AWSSession
import rasterio
import boto3

## Open AWS sessions

In [ ]:
out_dir = "/data/res/test_partial_product/"
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)

# use the creds created on CDSE website
with open("/data/creds_s3.json") as f:
    cred = json.load(f)

un = cred["username"]
pw = cred["password"]

# rasterio session
rio_session = AWSSession(
    aws_access_key_id=un,
    aws_secret_access_key=pw,
    region_name="default",
    endpoint_url="eodata.dataspace.copernicus.eu")

# needed for other (non-tiff) files
session = boto3.session.Session()
s3 = boto3.resource(
    's3',
    endpoint_url='https://eodata.dataspace.copernicus.eu',
    aws_access_key_id=un,
    aws_secret_access_key=pw,
    region_name='default'
)

## Search products with STAC API

In [ ]:
# Search using STAC api
catalog = Client.open(
    "https://stac.dataspace.copernicus.eu/v1/"
)
aoi_file = "../data/Morocco_AOI.geojson"
shp = gpd.read_file(aoi_file).geometry[0]
search = catalog.search(collections=["sentinel-1-slc"], intersects=shp)

# work only with the first result for now
it = next(search.items())

## Extract path info

In [ ]:
# product will be saved in this subdir
product_root_dir = f"{it.id}.SAFE"

# use manifest file to get S3 bucket and prefix
manifest_url = it.assets["safe_manifest"].href

# Parse the url
parsed = urlparse(manifest_url)
if parsed.scheme != "s3":
    raise ValueError("Product url does not start with s3://")

# Bucket is the "netloc"
bucket_name = parsed.netloc

# Look for subdir prefix
manifest_path = Path(parsed.path.lstrip("/"))
prefix = str(manifest_path.parent)

## List all files

In [ ]:
# try to find xml files
bucket = s3.Bucket(bucket_name)
files = [it.key for it in list(
    bucket.objects.filter(
        Prefix=prefix
    )
)]


In [ ]:
subdirs = ("annotation", "measurement", "preview", "support", "annotation/rfi")
# for subdir in subdirs:
    
# for f in files:
    # print(Path(f))


## Read burst ranges in raster

In [ ]:
# this may not be needed
url = it.assets["iw1-vv"].href
annotation_url = it.assets["schema-product-iw1-vh"].href

In [ ]:
# open remote dataset
with rasterio.Env(session=rio_session, AWS_VIRTUAL_HOSTING=False):
    ds = riox.open_rasterio(url, chunks="auto")

# download cropped array
# with ProgressBar():
    # ds[0, :1500].to_zarr("/data/res/test_s3_S1.zarr", mode="w")

This fails:
```bash
./sentinel1_burst_extractor_spatiotemporal.sh -p vv -s 2025-09-25 -e 2025-09-30 -x 30.19358 -y -7.2473372
```
Example from the docs also fails. Problem with string parsing.